<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/GenerateCloseEnoughPoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd


In [15]:
# Define the neural network
class FFNN(nn.Module):
    def __init__(self):
        super(FFNN, self).__init__()
        self.hidden = nn.Linear(2, 2)
        self.output = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x


# Custom loss function
def custom_loss(output, target):
    return -torch.mean(target * torch.log(output) + (1 - target) * torch.log(1 - output))


In [17]:
# Instantiate the network
model = FFNN()
# Optimizer (using simple Gradient Descent)
optimizer = optim.SGD(model.parameters(), lr=0.02)

# Dummy input and output data
input_data = torch.randn(10, 2)
print(input_data)
output_data = torch.randn(10, 1)
output_data = (output_data > 0.5).float()
# Training loop
max_num_epoch = 1000
prev_loss = None
for epoch in range(max_num_epoch):
    optimizer.zero_grad()   # clear the gradients
    outputs = model(input_data)  # forward pass
    loss = custom_loss(outputs, output_data)  # calculate the loss
    loss.backward()  # backward pass (compute gradients)
    optimizer.step()  # update weights

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.6f}')
        # Check for early stopping condition
    if prev_loss is not None:
        loss_diff = abs(prev_loss - loss.item())
        if loss_diff < 0.0001:
            print(f'Early stopping at epoch {epoch+1}, Loss: {loss.item():.6f}')
            break

    prev_loss = loss.item()

# Track weights and biases
weights_hidden = model.hidden.weight.data.numpy()
bias_hidden = model.hidden.bias.data.numpy()

weights_output = model.output.weight.data.numpy()
V_0_transposed = weights_output.T
bias_output = model.output.bias.data.numpy()


# Prepare data for DataFrame
input_data_list = [str(row.tolist()) for row in input_data]
output_data_list = [str(row.item()) for row in output_data]
weights_hidden_list = [str(row.tolist()) for row in weights_hidden]
bias_hidden_list = [str(bias_hidden.tolist())]
V_0_transposed_list = [str(row.tolist()) for row in V_0_transposed]
bias_output_list = [str(bias_output.tolist())]

# Create DataFrame
df = pd.DataFrame({
    'x_2dvec': input_data_list + [None] * (max(len(weights_hidden_list), len(V_0_transposed_list)) - len(input_data_list)),
    'y': output_data_list + [None] * (max(len(weights_hidden_list), len(V_0_transposed_list)) - len(output_data_list)),
    'W_0': weights_hidden_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(weights_hidden_list)),
    'b': bias_hidden_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(bias_hidden_list)),
    'V_0': V_0_transposed_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(V_0_transposed_list)),
    'c': bias_output_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(bias_output_list))
})

# Save to CSV file
df.to_csv('output.csv', index=False)

print("Weights of hidden layer:", weights_hidden)
print("Bias of hidden layer:", bias_hidden)
print("Weights of output layer:", weights_output)
print("Bias of output layer:", bias_output)


tensor([[-1.1959e+00, -1.7239e+00],
        [ 3.2780e-01, -9.2969e-04],
        [-3.2181e-01,  5.0979e-02],
        [ 5.1391e-02, -1.8637e+00],
        [ 1.4383e-01, -8.8913e-01],
        [ 8.2575e-01, -9.1271e-01],
        [-1.0502e+00,  2.1276e-01],
        [-3.4244e-01, -3.2497e+00],
        [ 5.9189e-01,  1.8070e+00],
        [-9.3555e-01,  1.2251e+00]])
Epoch [10/1000], Loss: 0.754942
Epoch [20/1000], Loss: 0.718320
Epoch [30/1000], Loss: 0.687161
Epoch [40/1000], Loss: 0.660629
Epoch [50/1000], Loss: 0.638005
Epoch [60/1000], Loss: 0.618680
Epoch [70/1000], Loss: 0.602139
Epoch [80/1000], Loss: 0.587948
Epoch [90/1000], Loss: 0.575744
Epoch [100/1000], Loss: 0.565223
Epoch [110/1000], Loss: 0.556131
Epoch [120/1000], Loss: 0.548253
Epoch [130/1000], Loss: 0.541411
Epoch [140/1000], Loss: 0.535453
Epoch [150/1000], Loss: 0.530254
Epoch [160/1000], Loss: 0.525704
Epoch [170/1000], Loss: 0.521715
Epoch [180/1000], Loss: 0.518208
Epoch [190/1000], Loss: 0.515118
Epoch [200/1000], Los

# 新段落